Copyright 2020 Konstantin Yakovlev, Matthias Anderer

   Licensed under the Apache License, Version 2.0 (the "License");
   you may not use this file except in compliance with the License.
   You may obtain a copy of the License at

       http://www.apache.org/licenses/LICENSE-2.0

   Unless required by applicable law or agreed to in writing, software
   distributed under the License is distributed on an "AS IS" BASIS,
   WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
   See the License for the specific language governing permissions and
   limitations under the License.

# Imports & Functions

In [1]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random
import pathlib
import lightgbm as lgb

# custom imports
from multiprocessing import Pool        # Multiprocess Runs
warnings.filterwarnings('ignore')

In [2]:
########################### Helpers
#################################################################################
## Seeder
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)


## Loss func hyperparamter: LOSS_MUTIPLIER

In [3]:
LOSS_MULTIPLIER = 1.0 # Set multiplier according to desired under-/overshooting

In [4]:
# define custom loss function
def custom_asymmetric_train(y_pred, y_true):
    y_true = y_true.get_label()
    residual = (y_true - y_pred).astype("float")
    grad = np.where(residual < 0, -2 * residual, -2 * residual * LOSS_MULTIPLIER)
    hess = np.where(residual < 0, 2, 2 * LOSS_MULTIPLIER)
    return grad, hess

# define custom evaluation metric
def custom_asymmetric_valid(y_pred, y_true):
    y_true = y_true.get_label()
    residual = (y_true - y_pred).astype("float")
    loss = np.where(residual < 0, (residual ** 2) , (residual ** 2) * LOSS_MULTIPLIER) 
    return "custom_asymmetric_eval", np.mean(loss), False

In [5]:
########################### Helper to load data by store ID
#################################################################################
# Read data
def get_data_by_store(store):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    # Leave only relevant store
    df = df[df['store_id']==store]
    
    ############
    # Create features list
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    # Skipping first n rows
    df = df[df['d']>=START_TRAIN].reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle(output_parent/f'test_{store_id}.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


# Customized Variables Def

In [6]:
########################### Model params
#################################################################################
lgb_params = {
        'boosting_type': 'gbdt',
        'objective': 'tweedie',
        'tweedie_variance_power': 1.1,
        'metric':'rmse',
        'n_jobs': -1,
        'seed': 42,
        'learning_rate': 0.2,
        'bagging_fraction': 0.85,
        'bagging_freq': 1, 
        'colsample_bytree': 0.85,
        'colsample_bynode': 0.85,
        #'min_data_per_leaf': 25,
        #'num_leaves': 200,
        'lambda_l1': 0.5,
        'lambda_l2': 0.5
}

In [7]:
########################### Vars
#################################################################################
VER = 1                          # Our model version
SEED = 42                        # We want all things
seed_everything(SEED)            # to be as deterministic 
lgb_params['seed'] = SEED        # as possible
N_CORES = psutil.cpu_count()     # Available CPU cores


#LIMITS and const
TARGET      = 'sales'            # Our target
START_TRAIN = 0                  # We can skip some rows (Nans/faster training)
END_TRAIN   = 1913+28            # End day of our train set
P_HORIZON   = 28                 # Prediction horizon
USE_AUX     = False               # Use or not pretrained models

#FEATURES to remove
## These features lead to overfit
## or values not present in test set
remove_features = ['id','state_id','store_id',
                   'date','wm_yr_wk','d',TARGET]

#PATHS for Features
ORIGINAL = pathlib.Path("./input/data")
input_parent = pathlib.Path("./input/fe_out")
BASE     = input_parent/'grid_part_1.pkl'
PRICE    = input_parent/'grid_part_2.pkl'
CALENDAR = input_parent/'grid_part_3.pkl'
output_parent = pathlib.Path("./input/bottom_out")

#STORES ids
STORES_IDS = pd.read_csv(ORIGINAL/'sales_train_validation.csv')['store_id']
STORES_IDS = list(STORES_IDS.unique())
print("stores ids are:", STORES_IDS)

stores ids are: ['CA_1', 'CA_2', 'CA_3', 'CA_4', 'TX_1', 'TX_2', 'TX_3', 'WI_1', 'WI_2', 'WI_3']


# Train Model

In [10]:
########################### Train Models
#################################################################################
for store_id in STORES_IDS:
    print('Train {}...'.format(store_id))
    
    # Get grid for current store: 1min
    # features_columns is ['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 
    # 'price_std', 'price_mean', 'price_norm', 'price_nunique', 'item_nunique', 'price_momentum', 
    # 'price_min', 'price_momentum_m', 'price_momentum_y', 'event_name_1', 'event_type_1', 
    # 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'tm_d', 'tm_w', 'tm_m', 
    # 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end']
    grid_df, features_columns = get_data_by_store(store_id)
    
    # Masks for 
    # Train (All data less than 1913)
    # "Validation" (Last 28 days - not real validatio set)
    # Test (All data greater than 1913 day, 
    #       with some gap for recursive features)
    train_mask = grid_df['d'] <= END_TRAIN
    valid_mask = train_mask & (grid_df['d'] > (END_TRAIN - P_HORIZON))
    preds_mask = grid_df['d'] > (END_TRAIN - 100)
    
    # Apply masks and save lgb dataset as bin
    # to reduce memory spikes during dtype convertations
    # https://github.com/Microsoft/LightGBM/issues/1032
    # "To avoid any conversions, you should always use np.float32"
    # or save to bin before start training
    # https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/53773
    train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                       label=grid_df[train_mask][TARGET])
    train_data.save_binary(output_parent/'train_data.bin')
    train_data = lgb.Dataset(output_parent/'train_data.bin')
    
    valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                       label=grid_df[valid_mask][TARGET])
    
    # Saving part of the dataset for later predictions
    # Removing features that we need to calculate recursively 
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]
    grid_df.to_pickle(output_parent/f'test_{store_id}.pkl')
    del grid_df
    
    # Launch seeder again to make lgb training 100% deterministic
    # with each "code line" np.random "evolves" 
    # so we need (may want) to "reset" it
    seed_everything(SEED)
    estimator = lgb.train(lgb_params,
                          train_data,
                          num_boost_round = 3600, # =n_estimators = num_trees
                          early_stopping_rounds = 50, 
                          valid_sets = [train_data, valid_data],
                          verbose_eval = 100,
                          fobj = custom_asymmetric_train
                          )
    
    # Save model - it's not real '.bin' but a pickle file
    # estimator = lgb.Booster(model_file='model.txt')
    # can only predict with the best iteration (or the saving iteration)
    # pickle.dump gives us more flexibility
    # like estimator.predict(TEST, num_iteration=100)
    # num_iteration - number of iteration want to predict with, 
    # NULL or <= 0 means use best iteration
    model_name = f'lgb_model_{store_id}_v{VER}.bin'
    pickle.dump(estimator, open(output_parent/model_name, 'wb'))

    # Remove temporary files and objects 
    # to free some hdd space and ram memory
    os.remove(output_parent/"train_data.bin")
    del train_data, valid_data, estimator
    gc.collect()
    
    # "Keep" models features for predictions
    MODEL_FEATURES = features_columns

Train CA_1...
[LightGBM] [Info] Saving data to binary file input\bottom_out\train_data.bin
[LightGBM] [Info] Load from binary file input\bottom_out\train_data.bin
[LightGBM] [Warning] Using self-defined objective function
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.487384 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 5643
[LightGBM] [Info] Number of data points in the train set: 4751349, number of used features: 29
[LightGBM] [Warning] Using self-defined objective function
Training until validation scores don't improve for 50 rounds
[100]	training's rmse: 2.56557	valid_1's rmse: 2.20605
[200]	training's rmse: 2.4639	valid_1's rmse: 2.13062
[300]	training's rmse: 2.40439	valid_1's rmse: 2.10047
[400]	training's rmse: 2.36267	valid_1's rmse: 2.07737
[500]	training's rmse: 2.33112	valid_1's rmse: 2.05069
[600]	training's rmse: 2.3

In [13]:
grid_df, features_columns = get_data_by_store(store_id)
grid_df[train_mask][features_columns]

,item_id,dept_id,cat_id,release,sell_price,price_max,price_min,price_std,price_mean,price_norm,...,snap_CA,snap_TX,snap_WI,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end
0,HOBBIES_1_004,HOBBIES_1,HOBBIES,0,4.339844,4.640625,4.339844,0.146118,4.523438,0.935547,...,0,0,0,26,8,2,0,4,5,1
1,HOBBIES_1_008,HOBBIES_1,HOBBIES,0,0.419922,0.500000,0.419922,0.020035,0.475830,0.839844,...,0,0,0,26,8,2,0,4,5,1
2,HOBBIES_1_009,HOBBIES_1,HOBBIES,0,1.559570,1.769531,1.559570,0.066345,1.720703,0.881348,...,0,0,0,26,8,2,0,4,5,1
3,HOBBIES_1_010,HOBBIES_1,HOBBIES,0,3.169922,3.169922,2.970703,0.028915,2.974609,1.000000,...,0,0,0,26,8,2,0,4,5,1
4,HOBBIES_1_012,HOBBIES_1,HOBBIES,0,5.980469,6.519531,5.980469,0.115967,6.468750,0.916992,...,0,0,0,26,8,2,0,4,5,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4736378,FOODS_3_823,FOODS_3,FOODS,0,2.980469,2.980469,2.480469,0.171631,2.800781,1.000000,...,0,0,0,22,20,5,5,4,6,1
4736379,FOODS_3_824,FOODS_3,FOODS,0,2.480469,2.679688,2.000000,0.253174,2.507812,0.925293,...,0,0,0,22,20,5,5,4,6,1
4736380,FOODS_3_825,FOODS_3,FOODS,0,3.980469,4.378906,3.980469,0.188599,4.117188,0.908691,...,0,0,0,22,20,5,5,4,6,1
4736381,FOODS_3_826,FOODS_3,FOODS,230,1.280273,1.280273,1.280273,0.000000,1.280273,1.000000,...,0,0,0,22,20,5,5,4,6,1


# Predict

In [14]:
########################### Predict
#################################################################################

# Create Dummy DataFrame to store predictions
all_preds = pd.DataFrame()

# Join back the Test dataset with 
# a small part of the training data 
# to make recursive features
base_test = get_base_test()
print("orginal base_test by combining all test_storeid.pkl")
display(base_test)

# Timer to measure predictions time 
main_time = time.time()

# Loop over each prediction day
# As rolling lags are the most timeconsuming
# we will calculate it for whole day
for PREDICT_DAY in range(1,29):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    # Make temporary grid to calculate rolling lags
    grid_df = base_test.copy()
        
    for store_id in STORES_IDS:
        
        # Read all our models and make predictions
        # for each day/store pairs
        model_name = f'lgb_model_{store_id}_v{VER}.bin' 
        if USE_AUX: # use pretrained models 
            model_name = AUX_MODEL + model_name
        
        estimator = pickle.load(open(output_parent/model_name, 'rb'))
        
        day_mask = base_test['d']==(END_TRAIN + PREDICT_DAY)
        store_mask = base_test['store_id']==store_id
        
        mask = (day_mask) & (store_mask)
        base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])
    
    # Make good column naming and add 
    # to all_preds DataFrame
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id', f'F{PREDICT_DAY}']
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()
        
    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    del temp_df
    
all_preds = all_preds.reset_index(drop=True)
print("After collection of predictions per stores and per")
display(all_preds)

orginal base_test by combining all test_storeid.pkl


,id,d,sales,item_id,dept_id,cat_id,release,sell_price,price_max,price_min,...,snap_TX,snap_WI,tm_d,tm_w,tm_m,tm_y,tm_wm,tm_dw,tm_w_end,store_id
0,HOBBIES_1_001_CA_1_evaluation,1842,4.0,HOBBIES_1_001,HOBBIES_1,HOBBIES,224,8.257812,9.578125,8.257812,...,1,0,13,6,2,5,2,5,1,CA_1
1,HOBBIES_1_002_CA_1_evaluation,1842,0.0,HOBBIES_1_002,HOBBIES_1,HOBBIES,20,3.970703,3.970703,3.970703,...,1,0,13,6,2,5,2,5,1,CA_1
2,HOBBIES_1_003_CA_1_evaluation,1842,1.0,HOBBIES_1_003,HOBBIES_1,HOBBIES,300,2.970703,2.970703,2.970703,...,1,0,13,6,2,5,2,5,1,CA_1
3,HOBBIES_1_004_CA_1_evaluation,1842,2.0,HOBBIES_1_004,HOBBIES_1,HOBBIES,5,4.640625,4.640625,4.339844,...,1,0,13,6,2,5,2,5,1,CA_1
4,HOBBIES_1_005_CA_1_evaluation,1842,5.0,HOBBIES_1_005,HOBBIES_1,HOBBIES,16,2.880859,3.080078,2.480469,...,1,0,13,6,2,5,2,5,1,CA_1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3902715,FOODS_3_823_WI_3_evaluation,1969,NaN,FOODS_3_823,FOODS_3,FOODS,0,2.980469,2.980469,2.480469,...,0,0,19,24,6,5,3,6,1,WI_3
3902716,FOODS_3_824_WI_3_evaluation,1969,NaN,FOODS_3_824,FOODS_3,FOODS,0,2.480469,2.679688,2.000000,...,0,0,19,24,6,5,3,6,1,WI_3
3902717,FOODS_3_825_WI_3_evaluation,1969,NaN,FOODS_3_825,FOODS_3,FOODS,0,3.980469,4.378906,3.980469,...,0,0,19,24,6,5,3,6,1,WI_3
3902718,FOODS_3_826_WI_3_evaluation,1969,NaN,FOODS_3_826,FOODS_3,FOODS,230,1.280273,1.280273,1.280273,...,0,0,19,24,6,5,3,6,1,WI_3


Predict | Day: 1
##########  0.38 min round |  0.38 min total |  39845.63 day sales |
Predict | Day: 2
##########  0.26 min round |  0.64 min total |  36546.00 day sales |
Predict | Day: 3
##########  0.32 min round |  0.97 min total |  37166.55 day sales |
Predict | Day: 4
##########  0.40 min round |  1.37 min total |  36662.94 day sales |
Predict | Day: 5
##########  0.33 min round |  1.70 min total |  41326.16 day sales |
Predict | Day: 6
##########  0.29 min round |  1.99 min total |  49308.41 day sales |
Predict | Day: 7
##########  0.29 min round |  2.28 min total |  49667.39 day sales |
Predict | Day: 8
##########  0.30 min round |  2.58 min total |  43635.76 day sales |
Predict | Day: 9
##########  0.27 min round |  2.86 min total |  37309.17 day sales |
Predict | Day: 10
##########  0.30 min round |  3.15 min total |  41456.85 day sales |
Predict | Day: 11
##########  0.27 min round |  3.43 min total |  42743.78 day sales |
Predict | Day: 12
##########  0.21 min round |  3.64

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_evaluation,0.753042,0.688188,0.688751,0.686679,0.768146,0.916658,0.874288,0.901520,0.744240,...,0.826474,0.927827,0.892749,0.766914,0.726412,0.720585,0.724602,0.799029,0.944285,0.716593
1,HOBBIES_1_002_CA_1_evaluation,0.258061,0.179185,0.179749,0.177677,0.265720,0.410616,0.366975,0.287745,0.235237,...,0.313481,0.411217,0.374869,0.268727,0.216486,0.210659,0.214676,0.296547,0.438186,0.371699
2,HOBBIES_1_003_CA_1_evaluation,0.380282,0.306656,0.300634,0.298563,0.386606,0.524676,0.481036,0.410726,0.363467,...,0.435126,0.526038,0.489690,0.391707,0.344716,0.332305,0.336321,0.418192,0.553006,0.486519
3,HOBBIES_1_004_CA_1_evaluation,1.576726,1.512710,1.513273,1.556411,1.859568,2.880035,3.226641,1.765071,1.568762,...,1.872590,2.845898,3.199796,1.546131,1.505628,1.499801,1.549028,1.846013,2.863223,2.817502
4,HOBBIES_1_005_CA_1_evaluation,1.019091,0.945465,0.946028,0.943957,1.054895,1.357839,1.305729,1.071401,1.002277,...,1.044939,1.300725,1.344590,1.057527,1.010536,1.004709,1.008726,1.112624,1.412312,1.343253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,FOODS_3_823_WI_3_evaluation,0.202623,0.143599,0.124301,0.099602,0.166274,0.314532,0.295167,0.156133,0.142843,...,0.221505,0.412340,0.473968,0.263467,0.253557,0.286319,0.184189,0.209371,0.353168,0.336223
30486,FOODS_3_824_WI_3_evaluation,0.158023,0.077515,0.058217,0.033519,0.086313,0.195474,0.181825,0.535540,0.076709,...,0.129575,0.354055,0.409779,0.185414,0.280990,0.313752,0.106136,0.125178,0.229878,0.209876
30487,FOODS_3_825_WI_3_evaluation,0.785415,0.726391,0.707093,0.680611,0.733405,0.828566,0.814917,0.661982,0.723801,...,0.879478,1.194102,1.227838,0.944876,1.039661,1.046878,0.820935,0.832240,0.948121,0.931320
30488,FOODS_3_826_WI_3_evaluation,1.121828,1.063728,1.044429,1.005161,0.955275,1.252617,1.109078,1.105437,1.092550,...,0.997041,1.338233,1.329249,1.159699,1.195015,1.216385,1.097569,1.013930,1.295008,1.367331


# Export Result

In [17]:
########################### Export
#################################################################################
# Reading competition sample submission and
# merging our predictions
# As we have predictions only for "_validation" data
# we need to do fillna() for "_evaluation" items
submission = pd.read_csv(ORIGINAL/'sample_submission.csv')[['id']]
submission = submission.merge(all_preds, on=['id'], how='left').fillna(0)
submission.to_csv(output_parent/f'submission_v{VER}.csv', index=False)